In [ ]:
#import numpy as np
#import scipy as sp
#import matplotlib as mpl
#import matplotlib.cm as cm
#import matplotlib.pyplot as plt
#import pandas as pd
#import time
#pd.set_option('display.width', 500)
#pd.set_option('display.max_columns', 100)
#pd.set_option('display.notebook_repr_html', True)

In [ ]:
'''
Based on SymSpell:

Originally written in C#:

// SymSpell: 1 million times faster through Symmetric Delete spelling correction algorithm
//
// The Symmetric Delete spelling correction algorithm reduces the complexity of edit candidate generation and dictionary lookup 
// for a given Damerau-Levenshtein distance. It is six orders of magnitude faster and language independent.
// Opposite to other algorithms only deletes are required, no transposes + replaces + inserts.
// Transposes + replaces + inserts of the input term are transformed into deletes of the dictionary term.
// Replaces and inserts are expensive and language dependent: e.g. Chinese has 70,000 Unicode Han characters!
//
// Copyright (C) 2015 Wolf Garbe
// Version: 3.0
// Author: Wolf Garbe <wolf.garbe@faroo.com>
// Maintainer: Wolf Garbe <wolf.garbe@faroo.com>
// URL: http://blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
// Description: http://blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
//
// License:
// This program is free software; you can redistribute it and/or modify
// it under the terms of the GNU Lesser General Public License, 
// version 3.0 (LGPL-3.0) as published by the Free Software Foundation.
// http://www.opensource.org/licenses/LGPL-3.0
//
// Usage: single word + Enter:  Display spelling suggestions
//        Enter without input:  Terminate the program
'''
pass

In [1]:
#import re, collections
import re

In [2]:
max_edit_distance = 3
dictionary = {}
longest_word_length = 0

In [3]:
def get_deletes_list(w):
    '''given a word, derive strings with up to max_edit_distance characters deleted'''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

In [ ]:
# get_deletes_list("tomorrow")

In [4]:
def create_dictionary_entry(w):
    '''add word and its derived deletions to dictionary'''
    # check if word is already in dictionary
    # dictionary entries are in the form: (list of suggested corrections, frequency of word in corpus)
    global longest_word_length
    new_real_word_added = False
    if w in dictionary:
        dictionary[w] = (dictionary[w][0], dictionary[w][1] + 1)  # increment count of word in corpus
    else:
        dictionary[w] = ([], 1)  
        longest_word_length = max(longest_word_length, len(w))
        
    if dictionary[w][1]==1:
        # first appearance of word in corpus
        # n.b. word may already be in dictionary as a derived word (deleting character from a real word)
        # but counter of frequency of word in corpus is not incremented in those cases)
        new_real_word_added = True
        deletes = get_deletes_list(w)
        for item in deletes:
            if item in dictionary:
                # add (correct) word to delete's suggested correction list if not already there
                if item not in dictionary[item][0]:
                    dictionary[item][0].append(w)
            else:
                dictionary[item] = ([w], 0)  # note frequency of word in corpus is not incremented
        
    return new_real_word_added

In [5]:
def create_dictionary(fname):
    word_count = 0
    print "Creating dictionary..." 
    
    with open(fname) as file:    
        for line in file:
            words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for word in words:
                if create_dictionary_entry(word):
                    word_count += 1
                    
    print "total unique words in corpus: %i" % word_count
    print "total items in dictionary (corpus words and deletions): %i" % len(dictionary)
    print "  edit distance for deletions: %i" % max_edit_distance
    print "  length of longest word in corpus: %i" % longest_word_length
        
    return dictionary

In [6]:
%%time
create_dictionary("/Users/K-Lo/Desktop/big.txt")
pass

Creating dictionary...
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 27 s, sys: 729 ms, total: 27.7 s
Wall time: 27.9 s


<div class="alert alert-info">
<b>For testing:</b> <p>
Can look up a specific entry in the dictionary below. <br>
shows (possible corrections, and frequency that entry itself is in corpus - 0 if not a real word) <br>
Note: will return key error if there are no corrections (because we are accessing dictionary directly here)
</div>

In [7]:
dictionary["essential"]

(['essentially', 'essentials'], 92)

In [8]:
dictionary["wack"]

(['wrack'], 0)

In [9]:
def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance between sequences.

    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

In [10]:
def get_suggestions(string, silent=False):
    '''return list of suggested corrections for potentially incorrectly spelled word'''
    if (len(string) - longest_word_length) > max_edit_distance:
        if not silent:
            print "no items in dictionary within maximum edit distance"
        return []
    
    # suggestions = []
    # s_dictionary = {}
    suggest_dict = {}
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        # print "processing '%s'" % q_item
        queue = queue[1:]
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus, and not already in suggestion list
            # so add to suggestion dictionary, indexed by the word with value (frequency in corpus, edit distance)
            # note q_items that are not the input string are shorter than input string 
            # since only deletes are added (unless manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = (dictionary[q_item][1], len(string) - len(q_item))
            
            ## the suggested corrections for q_item as stored in dictionary (whether or not
            ## q_item itself is a valid word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    # compute edit distance
                    # if len(sc_item)==len(q_item):
                    #    item_dist = len(string) - len(q_item)
                    # suggested items should always be longer (unless manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)
                    #elif len(q_item)==len(string):
                        # a suggestion could be longer or shorter than original string (bug in original FAROO?)
                        # if suggestion is from string's suggestion list, sc_item will be longer
                        # if suggestion is from a delete's suggestion list, sc_item may be shorter
                    #   item_dist = abs(len(sc_item) - len(q_item))
                    #else:
                    # check in original code, but probably not necessary because string has already checked
                    assert sc_item!=string
                    
                    # calculate edit distance using, for example, Damerau-Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    if item_dist<=max_edit_distance:
                        assert sc_item in dictionary  # should already be in dictionary if in suggestion list
                        suggest_dict[sc_item] = (dictionary[sc_item][1], item_dist)
        
        # now generate deletes (e.g. a substring of string or of a delete) from the queue item
        # as additional items to check -- add to end of queue
        assert len(string)>=len(q_item)
        if (len(string)-len(q_item))<max_edit_distance and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    q_dictionary[word_minus_c] = None  # arbitrary value, just to identify we checked this
             
    # queue is now empty: convert suggestions in dictionary to list for output
    
    if not silent:
        print "number of possible corrections: %i" %len(suggest_dict)
        print "  edit distance for deletions: %i" % max_edit_distance
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested corrections only:
    # return sorted(suggest_dict, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    as_list = suggest_dict.items()
    return sorted(as_list, key = lambda (term, (freq, dist)): (dist, -freq))

    '''
    Option 1:
    get_suggestions("file")
    ['file', 'five', 'fire', 'fine', ...]
    
    Option 2:
    get_suggestions("file")
    [('file', (5, 0)),
     ('five', (67, 1)),
     ('fire', (54, 1)),
     ('fine', (17, 1))...]  
    '''

<div class="alert alert-info">
<b>For testing:</b><p>
Type in word to correct below, to test and get whole list of possible suggestions.
</div>

In [11]:
%%time
get_suggestions("chacha")

number of possible corrections: 97
  edit distance for deletions: 3
CPU times: user 13.3 ms, sys: 5.18 ms, total: 18.4 ms
Wall time: 14.5 ms


[('coach', (26, 2)),
 ('chatham', (3, 2)),
 ('which', (4842, 3)),
 ('each', (411, 3)),
 ('change', (150, 3)),
 ('chair', (135, 3)),
 ('church', (117, 3)),
 ('chance', (96, 3)),
 ('reach', (85, 3)),
 ('charge', (70, 3)),
 ('coachman', (52, 3)),
 ('catch', (45, 3)),
 ('china', (43, 3)),
 ('check', (38, 3)),
 ('charm', (31, 3)),
 ('chain', (30, 3)),
 ('chancre', (30, 3)),
 ('chairs', (29, 3)),
 ('teach', (27, 3)),
 ('crack', (20, 3)),
 ('chase', (20, 3)),
 ('charcot', (19, 3)),
 ('chances', (18, 3)),
 ('clash', (16, 3)),
 ('couch', (15, 3)),
 ('chains', (14, 3)),
 ('chalk', (13, 3)),
 ('crash', (12, 3)),
 ('chart', (12, 3)),
 ('cash', (11, 3)),
 ('charcoal', (10, 3)),
 ('chap', (9, 3)),
 ('chat', (9, 3)),
 ('chanced', (8, 3)),
 ('chapel', (8, 3)),
 ('chaos', (8, 3)),
 ('masha', (8, 3)),
 ('hath', (7, 3)),
 ('chaps', (6, 3)),
 ('checks', (6, 3)),
 ('trachea', (5, 3)),
 ('chased', (5, 3)),
 ('hack', (4, 3)),
 ('hoch', (4, 3)),
 ('ache', (4, 3)),
 ('cava', (3, 3)),
 ('chateau', (3, 3)),
 ('c

In [13]:
%%time
#benchmark timing
for i in range(1000):
    get_suggestions("acamodation", silent=True)

CPU times: user 1.74 s, sys: 19.5 ms, total: 1.76 s
Wall time: 1.77 s


In [14]:
%%time
#benchmark timing
for i in range(1000):
    get_suggestions("acomodation", silent=True)

CPU times: user 2.1 s, sys: 14.1 ms, total: 2.11 s
Wall time: 2.13 s


In [15]:
%%time
#benchmark timing
for i in range(1000):
    get_suggestions("hous", silent=True)

CPU times: user 38 s, sys: 114 ms, total: 38.1 s
Wall time: 38.2 s


In [22]:
# get best word
def best_word(s, silent=False):
    try:
        return get_suggestions(s, silent)[0]
    except:
        return None

<div class="alert alert-info">
<b>For testing:</b><p>
Type in word to correct below, to test and get most suggested word.
</div>

In [23]:
best_word("hello")

number of possible corrections: 337
  edit distance for deletions: 3


('hello', (1, 0))

In [33]:
def correct_document(fname):
    with open(fname) as file:
        doc_word_count = 0
        corrected_word_count = 0
        unknown_word_count = 0
        print "Finding misspelled words in your document..." 
        
        for i, line in enumerate(file):
            doc_words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for doc_word in doc_words:
                doc_word_count += 1
                suggestion = best_word(doc_word, silent=True)
                if suggestion is None:
                    print "In line %i, the word %s was not found (no suggested correction)" % (i, doc_word)
                    unknown_word_count += 1
                elif suggestion[0]!=doc_word:
                    print "In line %i, %s: suggested correction is %s" % (i, doc_word, suggestion[0])
                    corrected_word_count += 1
        
    print "-----"
    print "total words checked: %i" % doc_word_count
    print "total potential errors found: %i" % corrected_word_count
    print "total unknown words: %i" % unknown_word_count
        
    return

<div class="alert alert-info">
<b>For testing:</b><p>
Provide test text file to correct, and give best suggestion (word level correction only) for errors.
</div>

In [34]:
correct_document("/Users/K-Lo/Desktop/test.txt")

Finding misspelled words in your document...
-----
total words checked: 8
total potential errors found: 0
total unknown words: 0


In [26]:
# from http://www.columbia.edu/acis/cria/rosenberg/sample/
correct_document("/Users/K-Lo/Desktop/OCRsample.txt")

Finding misspelled words in your document...
In line 3, taiths: suggested correction is faith
In line 11, the word oonipiittee was not found (no suggested correction)
In line 13, tj: suggested correction is to
In line 13, mnnff: suggested correction is snuff
In line 13, gjpt: suggested correction is get
In line 15, bh: suggested correction is by
In line 15, snc: suggested correction is sac
In line 15, uth: suggested correction is th
In line 15, unuer: suggested correction is under
In line 20, mthiitt: suggested correction is thirty
In line 21, cas: suggested correction is was
In line 22, pythian: suggested correction is scythian
In line 26, brainin: suggested correction is brain
In line 27, jfl: suggested correction is of
In line 28, ji: suggested correction is i
In line 28, stice: suggested correction is stick
In line 28, blaci: suggested correction is black
In line 28, eug: suggested correction is dug
In line 28, debbs: suggested correction is debts
In line 29, nericans: suggested co